<h1>Dataset Preprocessing to Enable Application of Classical ML Models</h1>

In [1]:
import json

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
from model.model import build_preprocessing

In [4]:
# Load configuration file from json in the given folder
with open("config.json", "r") as config_file:
    config = json.load(config_file)

In [5]:
# Set up the list of gestures
with open(config["Paths"]["Gesture list"], "r") as gesture_list:
    gestures = gesture_list.readlines()[0].split(", ")

In [29]:
img_size = config["General parameters"]["Image size"]

In [30]:
train_images, test_images = tf.keras.preprocessing.image_dataset_from_directory("Data",
                                                                                labels="inferred",
                                                                                label_mode="int",
                                                                                class_names=gestures,
                                                                                color_mode="rgb",
                                                                                batch_size=128,
                                                                                image_size=(img_size,
                                                                                            img_size),
                                                                                shuffle=True,
                                                                                seed=42,
                                                                                validation_split=0.2,
                                                                                subset="both")

Found 26351 files belonging to 49 classes.
Using 21081 files for training.
Using 5270 files for validation.


In [31]:
# Set the default preprocessing pipeline if not specified
preprocessing_layers = config["Model"]["Default preprocessing"]
#preprocessing_layers = "I,R"

# Build the preprocessing pipeline according to given instructions
preprocessing = build_preprocessing(inp_shape=[img_size,
                                               img_size,
                                               3],
                                    instructions=preprocessing_layers,
                                    name="preprocessing_pipeline")
img_size = 128
resize = tf.keras.layers.Resizing(img_size, img_size)

In [32]:
# Apply the same preprocessing steps as for the CNN but flatten the images
train_images = train_images.map(lambda x, y: (tf.reshape(resize(preprocessing(x)), [-1, img_size ** 2]), y)).unbatch()
test_images = test_images.map(lambda x, y: (tf.reshape(resize(preprocessing(x)), [-1, img_size ** 2]), y)).unbatch()

In [33]:
# Convert the tf.data.Datasets to numpy arrays containing image & label per sample
train_images = np.fromiter(tfds.as_numpy(train_images.take(10000)), dtype=tuple, count=10000)
test_images = np.fromiter(tfds.as_numpy(test_images.take(2000)), dtype=tuple, count=2000)

In [ ]:
# Extract the images and the labels separately
train_X, train_y = np.vstack([element[0] for element in train_images]), np.ravel(np.vstack([element[1] for element in train_images]))
test_X, test_y = np.vstack([element[0] for element in test_images]), np.ravel(np.vstack([element[1] for element in test_images]))

In [ ]:
# Check memory consumption for the training dataset
print("Training dataset memory consumption: {:.1f}MB".format(train_X.nbytes / (1024 * 1024.0)))

<h1>k-NN Gesture Classification</h1>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [ ]:
# Initialize and train the k-NN classifier
k = 3
knn = KNeighborsClassifier(n_neighbors=k,
                           n_jobs=-1)
knn.fit(train_X, train_y)

In [ ]:
# Evaluate the classifier on the test dataset
knn_predictions = knn.predict(test_X)

print(classification_report(test_y,
                            knn_predictions,
                            labels=gestures,
                            target_names=gestures))

In [ ]:
np.unique(knn_predictions, return_counts=True)

In [ ]:
knn_predictions

In [ ]:
np.unique(test_y, return_counts=True)